In [ ]:
#load whisper model
import stable_whisper
model = stable_whisper.load_model("large-v3")


In [2]:
import numpy as np
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip
from PIL import ImageFont, Image, ImageDraw

# 使用 PIL 創建包含字幕的圖像
def make_text_image(txt, size=(1280, 720), fontsize=36, font="msyh.ttc"):
    """
    使用 PIL 創建包含字幕的圖像
    """
    # 創建背景是透明的圖像
    img = Image.new('RGBA', size, color=(0, 0, 0, 0))  # 設置透明背景
    draw = ImageDraw.Draw(img)
    
    # 使用 TrueType 字型，大小可調
    font = ImageFont.truetype(font, fontsize)
    
    # 設置文本顏色
    text_color = (255, 255, 255)  # 白色
    
    # 計算文本邊界框大小（替代 textsize）
    bbox = draw.textbbox((0, 0), txt, font=font)
    text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]
    
    # 設置文本位置（居中）
    x = (size[0] - text_width) / 2
    y = size[1] - text_height - 20  # 距離底部20像素
    
    # 寫入文本
    draw.text((x, y), txt, font=font, fill=text_color)
    
    # 將 PIL 圖像轉換為 NumPy 數組，以便 MoviePy 使用
    img_array = np.array(img)
    
    # 返回 ImageClip 對象，這樣 MoviePy 才能處理
    return ImageClip(img_array).set_duration(3).set_position(("center", "bottom"))

# 解析 TSV 文件
def parse_tsv(tsv_path):
    """
    解析 TSV 字幕文件，將其轉換為適用於影片的字幕 (start, end, text) 格式
    """
    subtitles = []
    with open(tsv_path, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines()
        for line in lines:
            parts = line.split('\t')  # 分割行
            if len(parts) >= 3:
                start_time = int(parts[0]) / 1000.0  # 轉換毫秒為秒
                end_time = int(parts[1]) / 1000.0  # 轉換毫秒為秒
                text = parts[2]
                subtitles.append(((start_time, end_time), text))
    return subtitles

# 轉換 chunks 進行字幕處理
def create_subtitles_from_chunks(chunks):
    subtitles = []
    for chunk in chunks:
        start_time, end_time = chunk["timestamp"]
        text = chunk["text"]
        subtitles.append(((start_time, end_time), text))
    return subtitles

def add_subtitles_to_video(video_path, tsv_path, output_path):
    video = VideoFileClip(video_path)

    # 解析 TSV 字幕文件
    subtitles = parse_tsv(tsv_path)
    
    subtitle_clips = []
    
    for (start, end), text in subtitles:
        subtitle_clip = make_text_image(text)
        subtitle_clip = subtitle_clip.set_start(start).set_end(end)
        subtitle_clips.append(subtitle_clip)
    
    # 使用 CompositeVideoClip 合成視頻和字幕
    final_video = CompositeVideoClip([video] + subtitle_clips)
    
    # 將帶有字幕的影片寫出
    final_video.write_videofile(output_path, codec="libx264", fps=24)


In [ ]:
song = r"C:\Users\quant\Desktop\whisper\lemon.mp4"
result = model.transcribe(song)
result.to_tsv('audio.tsv')

In [5]:
import time
from googletrans import Translator

# 創建一個 Translator 物件
translator = Translator()

def trans_tsv(filename : str , src : str , dest : str):
    with open(filename,"r") as f:
        lines = f.readlines()
    
    with open(filename[:-4] + "_" + src + "_2_" + dest + ".tsv","w") as f:
        for line in lines:
            if len(line.strip().split()) != 0:
                new_line = line.strip().split()[0] + "\t" + line.strip().split()[1] + "\t" + translator.translate("".join(line.strip().split()[2:]), src=src, dest=dest).text + "\n"
                f.write(new_line)
                print(new_line)
                # time.sleep(3)
            else:
                f.write("\n")

    return filename[:-4] + "_" + src + "_2_" + dest + ".tsv","w"


trans_tsv("audio.tsv", "ja" , "zh-TW")

8720	14040	夢想有多好

14220	20040	我仍然夢見你

20440	25180	回來找一個被遺忘的人

25720	29920	一個舊的記憶

30420	31320	支付

33390	39260	很高興回來

39340	44720	終於你告訴我

45100	50320	即使是黑暗的過去也沒有說

50480	55820	如果您不這樣做，永遠天黑

56080	66800	我知道我不會再受傷了

67060	72660	即使是那天的悲傷，甚至那天的痛苦

72680	77940	和你愛一切的你在一起

78060	83640	這種氣味似乎不留在你的胸部

84080	93860	我無法更改它，直到雨水停止，您仍然有我的光

105570	110740	在黑暗中追踪你的背

111040	116460	我清楚地記得大綱

116780	122020	每次我遇到我無法接受的東西

122240	124860	我忍不住溢出

124980	127460	現在只有眼淚

127580	131040	你在做什麼？

131240	133700	你在看什麼？

134040	138600	我不知道的個人資料

138840	141460	你現在

141600	144360	與我相似

144400	149300	如果你處於眼淚的孤獨狀態

149800	154440	請忘記我，等等。

155440	160520	我真誠地希望

160780	165560	你還有我的光

168150	179900	對戀愛的人比你想像的

180260	190740	我不能那樣呼吸

190940	196580	我在附近

197720	200640	就像謊言

202360	206840	我不能忘記

207560	213780	這是真的

218120	221540	甚至那天的悲傷

221660	224260	甚至那天的痛苦

224420	229640	和你愛一切的你在一起

229860	235540	不留在胸部的苦檸檬的氣味

235540	240240	我不能更改它，直到下雨

240740	246080	像切割水果之一

246440	250880	你還有我的光



('audio_ja_2_zh-TW.tsv', 'w')

In [6]:
video_path = r"C:\Users\quant\Desktop\whisper\lemon.mp4"
tsv_path = r"C:\Users\quant\Desktop\whisper\audio_ja_2_zh-TW.tsv"
output_path = r"C:\Users\quant\Desktop\whisper\output_with_subtitles.mp4"

add_subtitles_to_video(video_path, tsv_path, output_path)

Moviepy - Building video C:\Users\quant\Desktop\whisper\output_with_subtitles.mp4.
MoviePy - Writing audio in output_with_subtitlesTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C:\Users\quant\Desktop\whisper\output_with_subtitles.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\quant\Desktop\whisper\output_with_subtitles.mp4
